# AI534 IA 4: Exploring word embeddings

**Overview**:
In this assignment, you will explore pre-trained word embeddings and use them to better understand textual data and improve downstream models.

Specifically, you will:
* Build a compact dataset of semantically related words using GloVe embeddings
* Cluster the word embeddings and compare different clustering metrics
* Visualize clusters using PCA and t-SNE
* (Bonus) Incorporate word embeddings or word-cluster representations into sentiment classification, building on your work in IA3

**What you need to submit**:
1. Your completed notebook in `.ipynb` format.
2. A PDF report that includes all code outputs and figures.

If some figures or outputs are missing in the PDF due to rendering/scrolling issues, manually add them (e.g., by inserting screenshots or exporting via another tool) so your PDF has all required results.

We have supplied auxiliary code for working with word embeddings. It is advisable to retain this code in its original form. Should you opt to modify this helper code, please ensure that your alterations are accompanied by comprehensive comments. This will facilitate your TA's understanding of the modifications and the rationale behind them.

In [ ]:
!pip install nbconvert > /dev/null 2>&1
!pip install pdfkit > /dev/null 2>&1
!apt-get install -y wkhtmltopdf > /dev/null 2>&1
import os
import pdfkit
import contextlib
import sys
from google.colab import files
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import math
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
# add more imports if necessary

# Background: Data and Word Embeddings

For the first part of this assignment, you will use GloVe, a word embedding model pre-trained on large corpora of unlabeled text. There are many other word embedding methods (see, for example, this overview: https://www.turing.com/kb/guide-on-word-embeddings-in-nlp), but here we will use GloVe embeddings (https://nlp.stanford.edu/projects/glove/).

Conceptually, in Part 1 you will treat words as the objects of interest and their GloVe embeddings as feature vectors describing them. These embeddings place words as points in a continuous “semantic” space, where semantically similar terms (such as *good* and *nice*) are located close to one another.

To avoid dealing with the full GloVe vocabulary, we provide a reduced file `GloVe_Embedder_data.txt` on Canvas. This file contains a subset of words from the IA3 sentiment dataset that also appear in the full GloVe vocabulary, along with their embeddings. Download this file and place it in the same Google Drive directory as the rest of your AI534 data so the notebook can access it.

If a word you encounter is not present in this embedding file, it will simply be treated as an unknown token by the helper code. This is expected behavior and does not require any special handling from you.

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
EMBEDDING_PATH = '/content/gdrive/My Drive/AI534/GloVe_Embedder_data.txt' #please do not modify this path

Mounted at /content/gdrive


In [ ]:
# Helper class and functions --- Please leave as is.
# If you need to modify this block, please clearly indicate your change by providing detailed comments.
#
# Loads GloVe embeddings from a designated file location.
#
# Invoked via:
# ge = GloVe_Embedder(path_to_embeddings)
#
# Embed single word via:
# embed = ge.embed_str(word)
#
# Embed a list of words via:
# embeds = ge.embed_list(word_list)
#
# Find k nearest neighbors of word via:
# ge.find_k_nearest(word, k)
#
# Save vocabulary to file via:
# ge.save_to_file(path_to_file)

class GloVe_Embedder:
    def __init__(self, path):
        self.embedding_dict = {}
        self.embedding_array = []
        self.unk_emb = 0
        # Adapted from https://stackoverflow.com/questions/37793118/load-pretrained-GloVe-vectors-in-python
        with open(path,'r') as f:
            for line in f:
                split_line = line.split()
                word = split_line[0]
                embedding = np.array(split_line[1:], dtype=np.float64)
                self.embedding_dict[word] = embedding
                self.embedding_array.append(embedding.tolist())
        self.embedding_array = np.array(self.embedding_array)
        self.embedding_dim = len(self.embedding_array[0])
        self.vocab_size = len(self.embedding_array)
        self.unk_emb = np.zeros(self.embedding_dim)

    # Check if the provided embedding is the unknown embedding.
    def is_unk_embed(self, embed):
        return np.sum((embed - self.unk_emb) ** 2) < 1e-7

    # Check if the provided string is in the vocabulary.
    def token_in_vocab(self, x):
        if x in self.embedding_dict and not self.is_unk_embed(self.embedding_dict[x]):
            return True
        return False

    # Returns the embedding for a single string and prints a warning if
    # the string is unknown to the vocabulary.
    #
    # If indicate_unk is set to True, the return type will be a tuple of
    # (numpy array, bool) with the bool indicating whether the returned
    # embedding is the unknown embedding.
    #
    # If warn_unk is set to False, the method will no longer print warnings
    # when used on unknown strings.
    def embed_str(self, x, indicate_unk = False, warn_unk = True):
        if self.token_in_vocab(x):
            if indicate_unk:
                return (self.embedding_dict[x], False)
            else:
                return self.embedding_dict[x]
        else:
            if warn_unk:
                    print("Warning: provided word is not part of the vocabulary!")
            if indicate_unk:
                return (self.unk_emb, True)
            else:
                return self.unk_emb

    # Returns an array containing the embeddings of each vocabulary token in the provided list.
    #
    # If include_unk is set to False, the returned list will not include any unknown embeddings.
    def embed_list(self, x, include_unk = True):
        if include_unk:
            embeds = [self.embed_str(word, warn_unk = False).tolist() for word in x]
        else:
            embeds_with_unk = [self.embed_str(word, indicate_unk=True, warn_unk = False) for word in x]
            embeds = [e[0].tolist() for e in embeds_with_unk if not e[1]]
            if len(embeds) == 0:
                print("No known words in input:" + str(x))
                embeds = [self.unk_emb.tolist()]
        return np.array(embeds)

    # Finds the vocab words associated with the k nearest embeddings of the provided word.
    # Can also accept an embedding vector in place of a string word.
    # Return type is a nested list where each entry is a word in the vocab followed by its
    # distance from whatever word was provided as an argument.
    def find_k_nearest(self, word, k, warn_about_unks = True):
        if type(word) == str:
            word_embedding, is_unk = self.embed_str(word, indicate_unk = True)
        else:
            word_embedding = word
            is_unk = False
        if is_unk and warn_about_unks:
            print("Warning: provided word is not part of the vocabulary!")

        all_distances = np.sum((self.embedding_array - word_embedding) ** 2, axis = 1) ** 0.5
        distance_vocab_index = [[w, round(d, 5)] for w,d,i in zip(self.embedding_dict.keys(), all_distances, range(len(all_distances)))]
        distance_vocab_index = sorted(distance_vocab_index, key = lambda x: x[1], reverse = False)
        return distance_vocab_index[:k]

    def save_to_file(self, path):
        with open(path, 'w') as f:
            for k in self.embedding_dict.keys():
                embedding_str = " ".join([str(round(s, 5)) for s in self.embedding_dict[k].tolist()])
                string = k + " " + embedding_str
                f.write(string + "\n")

# Part 0(10 pts) : Build your data set of words for exploration.
In this part you will be a small data set of words and explore both clustering and dimension reduction on this data.

##🚧 Task: Build your own data set of words.
You will begin by constructing a compact dataset of words for visualization and experimentation.
Use the following seed words as your starting point:

**`flight`, `awesome`, `terrible`, `help`, `late`**

For each seed word:

* Retrieve the **30 nearest neighbor words** from the provided vocabulary (`GloVe_Embedder_data.txt`).
* Use the `find_k_nearest` function (Euclidean distance) to retrieve its nearest neighbors.  
  **Note:** `find_k_nearest` will return the seed word itself as one of the neighbors. Thus you can request **31 neighbors**, then **remove the seed word** from the result to obtain the desired **30 nearest neighbor words**.
* Store both the retrieved words and their embeddings.

If a word appears in the neighbor lists of multiple seed words, keep **one copy** of that word in your final dataset and assign it to the seed word to which it is **closest** (smallest distance).

This will give you **up to 150 unique words** (fewer if there is overlap), grouped into five clusters corresponding to the five seed words.

As a reference, display the 30 nearest neighbors for each seed word in a **dataframe**.


In [ ]:
# Your code goes here


# Part 1 (35pts): Clustering the words

##🚧 (15 pts) Task 1.1: Kmeans objective as a function of $k$
Apply the k-means clustering algorithm to your word-embedding dataset using a range of cluster counts.  
Use `sklearn.cluster.KMeans` and keep the default settings **except** for `n_clusters`.

For each value of \(k\) from **2 to 20**:

* Fit a k-means model to your word embeddings.
* Record the **k-means objective** (the `inertia_` attribute in scikit-learn), defined as  
  $$
  \sum_{i=1}^k \sum_{x \in C_i} \|x - \mu_i\|^2 .
  $$

Finally, **plot the k-means objective as a function of \(k\)**.  
Include clear axis labels and a descriptive title.


In [ ]:
# Your code goes here

##✍️ **Question:**
1. Is the k-means objective strictly decreasing as \(k\) increases? In practice you may observe some deviations from monotonic decrease, explain why they may occur?

2. Does the plotted curve provide any evidence that \(k = 5\) might be a reasonable number of clusters for this dataset? Discuss your reasoning.

**Your answer goes here.**

## 🚧 (20 pts) Task 1.2: evaluating your clustering against ground truth (20 pts)
Use the original seed-word assignments as **ground-truth labels** for your dataset, and evaluate the quality of your k-means clustering results for different values of \(k\).
**Reminder:** The ground-truth label for each word is the seed word whose neighbor list it came from (after resolving any overlaps using the distance rule from Task 1.1).
For each \(k\) from **2 to 20**, compute the following metrics:

- **Purity** (You must implement this metric yourself.)
- **Adjusted Rand Index (ARI)**  Use `sklearn.metrics.adjusted_rand_score`.
- **Normalized Mutual Information (NMI)** Use `sklearn.metrics.normalized_mutual_info_score`.

Plot each metric as a function of \(k\).  
Be sure to label axes clearly and provide legends where appropriate.





In [ ]:
# Your code goes here.

##✍️ **Question:**

1. Based on the three evaluation metrics (Purity, ARI, NMI), does \(k = 5\) appear to give the best scores? Comment on how each metric behaves as \(k\) varies, and explain why the “best” value of \(k\) may or may not align with the true number of clusters in the data.

2. Suppose you want to compare **two different clustering algorithms**, each of which automatically chooses its own number of clusters (for example, one may return 5 clusters while the other returns 10). Among Purity, ARI, and NMI, which metric(s) are appropriate for comparing their performance?  Explain your reasoning—particularly how each metric handles differences in the number of clusters.

**Your answer goes here.**


# Part 2 (35 pts): Dimension reduction and visualization
In this part, you will reduce the dimension of your data to 2-d using PCA and t-SNE and visualize them.

## 🚧  (15 pts) Task 2.1: apply PCA dimension reduction

1. Apply **Principal Component Analysis (PCA)** to your word-embedding dataset using  
`sklearn.decomposition.PCA` and project the embeddings into **2 dimensions**.

2. Create a **2D scatter plot** of the PCA-reduced embeddings using `matplotlib`, using **different colors** to indicate the cluster associated with each seed word (i.e., the seed word from which each neighbor originated).
4. Annotate a **selected set of words**—including some from well-separated regions and some from visually overlapping regions—so the plot is easier to interpret. Use the `annotate` function from `matplotlib` to label the chosen points.

Your plot should include:
* Axis labels (e.g., “PC1”, “PC2”)  
* A legend mapping colors to seed-word clusters  
* A descriptive title

In [ ]:
# Your code goes here.

##✍️ **Question**: Reflection on PCA results
Does the 2-D PCA visualization reveal **five clearly separated clusters**?  
Describe where clusters appear well-separated and where they overlap or mix.  
In your discussion, consider that both the underlying word embeddings **and** the PCA projection to 2-D can influence the amount of visible separation.

**Your answer goes here:**


## 🚧(20 pts) Task 2.2: apply t-SNE

Now apply **t-SNE**, a nonlinear dimensionality-reduction method, to your word-embedding dataset.  
Use `sklearn.manifold.TSNE` with **Euclidean distance** to project the embeddings into **2 dimensions**.

t-SNE is sensitive to its **perplexity** parameter. The original authors recommend values between 5 and 50, so for this assignment you should run t-SNE using the following perplexities:

**5, 10, 20, 30, 40, 50**

For each chosen perplexity:

1. Compute the 2-D t-SNE embedding.  
2. Create a **2D scatter plot** using the same color scheme as in the PCA task (one color per seed-word cluster).  
3. Annotate a **selected set of words** that help you interpret the visualization—include examples from **well-separated** regions and **visually overlapping** regions.  
   Use the `annotate` function from `matplotlib` for labeling points.

Your plots should include:
* Axis labels (e.g., “t-SNE dim 1”, “t-SNE dim 2”)  
* A legend mapping colors to seed-word clusters  
* A descriptive title indicating the perplexity used

In [ ]:
# Your code goes here

##✍️ **Question: Reflection on t-SNE results**

Across the different perplexity settings, do the 2-D t-SNE visualizations reveal distinct clusters?
Describe where clusters appear well separated and where they overlap or mix, and how this changes with perplexity.
Comment on how the perplexity parameter influences the visible structure.

 **Your answe goes here.**

##✍️ Question: PCA vs. t-SNE
Compare your PCA and t-SNE visualizations.
Which method provides clearer separation between clusters, and in what ways do their results differ?
Discuss how PCA’s linear projection and t-SNE’s emphasis on local neighborhoods may explain the differences you observe.

# Optional Part 3 (Up to 30 bonus pts): Using word embeddings to improve tweet classification

In this bonus task, you will return to the **sentiment classification dataset** used in IA3 and explore how to improve the basic bag-of-words (BoW) representation by leveraging **word embeddings**.

Your goal is to answer the following question:

> **How can word embeddings be used to create a more effective representation of tweets for classification?**

You should continue using classifiers covered in this course (e.g., logistic regression, naive Bayes, SVM, k-NN, tree-based models).  
**Do not use deep learning models** for this bonus task.

Because the dataset is heavily imbalanced (≈80% negative, ≈20% positive), use **Area Under the ROC Curve (AUC)** (`sklearn.metrics.roc_auc_score`) on the validation data as your primary performance metric.



## 🌱 Seed ideas.

Below are two example directions. You may explore one of these or propose your own.

---

### **Idea 1: Embedding-based averaged representations**

Represent a tweet as the **weighted average** of its word embeddings.  
For example:
- Weight each word via its (normalized) tf-idf value.
- Average the weighted embeddings to obtain a dense, low-dimensional vector.

This can reduce dimensionality and potentially improve generalization.

---

### **Idea 2: Bag-of-word-clusters (with optional bi-clusters)**

BoW treats semantically similar words as unrelated.  
To reduce redundancy:

- Cluster the vocabulary based on word embeddings (e.g., k-means).
- Replace each word in a tweet with its **cluster ID** to form a bag-of-cluster representation.

**Optional extension:**  
Consider the benefit of bigram features in IA3, you could also consider mapping bigrams to **bi-clusters**:
- For a bigram *(w₁, w₂)*, replace it with the pair *(cluster(w₁), cluster(w₂))*.
- Build a bag-of-bi-clusters representation.

This preserves some local structure while greatly reducing feature dimensionality.

---


## 💯 Bonus Point Structure (up to 30 points)
Your bonus points will be awarded in three tiers:

#### **Tier 1 — Basic implementation (10 pts)**  
Awarded if you:
- Implement one valid idea correctly.
- Train at least one classifier on your new representation.
- Report training and validation AUC.
- Provide a brief description of your approach.

#### **Tier 2 — Thoughtful exploration (10 pts)**  
Awarded for going beyond the basics by doing **one or more** of:
- Exploring meaningful hyperparameter variations  
  (e.g., number of clusters, tf-idf weighting schemes, classifier choices).
- Providing informative comparisons or plots.
- Offering a clear interpretation of overfitting/underfitting behavior.
- Trying multiple variants within your chosen idea.

#### **Tier 3 — Insightful extension (10 pts)**  
Awarded for deeper insight or creative extensions, such as:
- Incorporating **bi-clusters** or combining embedding-based ideas in a justified way.
- Providing a well-reasoned explanation of how the representation influences classifier behavior.
- Discussing limitations or failure modes.
- Presenting a well-organized, thoughtful mini-report.

**Total possible bonus: 30 points**



## 🚧 What to do

Choose **one main idea** and explore it.  
You may try different variants of your chosen idea (e.g., number of clusters, weighting schemes, classifiers), and you **may** explore more than one idea if you wish, **but this is not required**.  As indicated by the bonus point struture, the bonus credit is awarded based on the **quality and depth** of your exploration, not the number of ideas attempted.

You may use any classifier covered in this course.

You are **not required** to achieve improved validation performance compared to IA3.

Include your code (below) in this notebook and provide a brief report.
---

In [ ]:
# Your code goes here


## ✍️ Bonus-part report.
Your report should:

1 Describes your idea (including any hyperparameters or variants you explored).

2. Summarizes your results (training and validation performance, using AUC as the main metric).

3. Interprets the results. For example: How did your representation affect overfitting/underfitting compared to BoW? Did the embedding-based features change the classifier’s behavior or sensitivity?

Your report goes here.

In [ ]:
#running this code block will convert this notebook and its outputs into a pdf report.
!jupyter nbconvert --to html /content/gdrive/MyDrive/Colab\ Notebooks/IA4-2024.ipynb  # you might need to change this path to appropriate value to location your copy of the IA0 notebook

input_html = '/content/gdrive/MyDrive/Colab Notebooks/IA4-2024.html' #you might need to change this path accordingly
output_pdf = '/content/gdrive/MyDrive/Colab Notebooks/IA4output.pdf' #you might need to change this path or name accordingly

# Convert HTML to PDF
pdfkit.from_file(input_html, output_pdf)

# Download the generated PDF
files.download(output_pdf)